<a href="https://colab.research.google.com/github/bingronglee/SillyCode/blob/main/doc_scraper_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 公文系統爬蟲工具
自動登入公文系統並爬取收文/發文資料,輸出為 Excel 檔案

## 🔧 參數設定區

In [1]:
# ==================== 登入帳號 ====================
USERNAME = "0928630343"  #@param {type:"string"}
PASSWORD = "780411"  #@param {type:"string"}

# ==================== 查詢條件 ====================
HANDLER_NAME = "蔡秉融"  #@param {type:"string"}
NUM_DOCS = 15  #@param {type:"slider", min:1, max:100, step:1}

# ==================== 日期篩選 ====================
USE_DATE_FILTER = True  #@param {type:"boolean"}
START_DATE = "2025-01-01"  #@param {type:"date"}
END_DATE = "2025-02-22"  #@param {type:"date"}

print(f"✅ 設定完成 | 帳號:{USERNAME} | 承辦人:{HANDLER_NAME or '不限'} | 數量:{NUM_DOCS}")
if USE_DATE_FILTER:
    print(f"   日期範圍: {START_DATE} ~ {END_DATE}")

✅ 設定完成 | 帳號:0928630343 | 承辦人:蔡秉融 | 數量:15
   日期範圍: 2025-01-01 ~ 2025-02-22


## 📦 安裝套件

In [8]:
# 安裝必要套件
!pip install -q selenium webdriver-manager openpyxl

# 安裝 Google Chrome 瀏覽器
!apt-get update > /dev/null 2>&1
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb > /dev/null 2>&1
!dpkg -i google-chrome-stable_current_amd64.deb > /dev/null 2>&1
!apt-get install -yf > /dev/null 2>&1 # 安裝可能缺少的依賴
!rm google-chrome-stable_current_amd64.deb

print("✅ 套件安裝完成")

✅ 套件安裝完成


## 🛠️ 工具函數

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re, openpyxl
from datetime import datetime
from collections import defaultdict

def setup_driver():
    opts = Options()
    opts.add_argument('--headless=new')
    opts.add_argument('--no-sandbox')
    opts.add_argument('--disable-dev-shm-usage')
    opts.add_argument('--disable-gpu')
    opts.add_argument('--log-level=3')
    opts.add_argument('--disable-blink-features=AutomationControlled')
    # 使用 webdriver-manager 自動安裝和管理 ChromeDriver
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=opts)

def get_links(driver, n):
    wait = WebDriverWait(driver, 10)
    links, page = [], 1
    while len(links) < n:
        try:
            table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'list_table')))
            rows = table.find_elements(By.TAG_NAME, 'tr')[1:]
            page_links = []
            for row in rows:
                try:
                    href = row.find_elements(By.TAG_NAME, 'td')[-1].find_element(By.TAG_NAME, 'a').get_attribute('href')
                    page_links.append(href.split('doc_id=')[1])
                except: pass
            links.extend(page_links)
            print(f"第{page}頁: {len(page_links)}個")
            if len(links) >= n or not page_links: break
            next_btn = driver.find_elements(By.XPATH, "//a[text()='下一頁']")
            if next_btn:
                next_btn[0].click()
                page += 1
                wait.until(EC.staleness_of(table))
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'list_table')))
            else: break
        except Exception as e:
            print(f"錯誤: {e}")
            break
    return links[:n]

def check_date(doc_date, start, end):
    if not start: return True
    try:
        parts = doc_date.split('/')
        dt = datetime(int(parts[0])+1911, int(parts[1]), int(parts[2]))
        return datetime.combine(start, datetime.min.time()) <= dt <= datetime.combine(end, datetime.max.time())
    except: return False

def extract_info(driver, doc_id, is_out=False):
    url = f"http://www.landsen.com.tw/lsdoc/index.php?pid=DOC04{'&page_view=B' if is_out else ''}&item=view&doc_id={doc_id}"
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
    info = {}
    try:
        if is_out:
            info['日期'] = driver.find_element(By.XPATH, "//td[contains(text(),'發文日期')]/b").text.strip()
            info['工地別'] = re.sub(r'【.*?】', '', driver.find_element(By.XPATH, "//th[contains(text(),'工地別')]/following-sibling::td").text).strip()
            info['發文字號'] = driver.find_element(By.XPATH, "//td[contains(text(),'發文字號')]").text.replace("發文字號:", "").strip()
            info['主旨'] = driver.find_element(By.XPATH, "//td[@valign='TOP' and @style='font-size:15px;line-height:150%;']").text.strip()
        else:
            info['公文日期'] = driver.find_element(By.XPATH, "//th[contains(text(),'公文日期')]/following-sibling::td").text.strip()
            info['工地別'] = re.sub(r'【.*?】', '', driver.find_element(By.XPATH, "//th[contains(text(),'工地別')]/following-sibling::td").text).strip()
            info['公文字號'] = driver.find_element(By.XPATH, "//th[contains(text(),'公文字號')]/following-sibling::td").text.strip()
            info['主旨摘要'] = driver.find_element(By.XPATH, "//th[contains(text(),'主旨摘要')]/following-sibling::td/div").text.strip()
    except Exception as e:
        print(f"提取失敗 {doc_id}: {e}")
        return None
    return info

def make_excel(data, path='/content/公文整理.xlsx'):
    wb = openpyxl.Workbook()
    wb.remove(wb.active)
    border = openpyxl.styles.Border(left=openpyxl.styles.Side(style='thin'),
                                     right=openpyxl.styles.Side(style='thin'),
                                     top=openpyxl.styles.Side(style='thin'),
                                     bottom=openpyxl.styles.Side(style='thin'))
    for site in sorted(data.keys()):
        ws = wb.create_sheet(title=site)
        row = 1
        incoming = [r for r in data[site] if r['type']=='收文']
        if incoming:
            for col, val in [('A1','公文日期'),('B1','公文字號'),('C1','主旨摘要')]:
                ws[col] = val
                ws[col].border = border
            row = 2
            for r in incoming:
                ws[f'A{row}'] = r['date']
                ws[f'B{row}'] = r['number'].replace('字','字\n') if '字' in r['number'] else r['number']
                ws[f'C{row}'] = r['subject']
                for c in ['A','B','C']: ws[f'{c}{row}'].border = border
                row += 1
        row += 1
        outgoing = [r for r in data[site] if r['type']=='發文']
        if outgoing:
            for col, val in [(f'A{row}','日期'),(f'B{row}','發文字號'),(f'C{row}','主旨')]:
                ws[col] = val
                ws[col].border = border
            row += 1
            for r in outgoing:
                ws[f'A{row}'] = r['date']
                ws[f'B{row}'] = r['number'].replace('字','字\n') if '字' in r['number'] else r['number']
                ws[f'C{row}'] = r['subject']
                for c in ['A','B','C']: ws[f'{c}{row}'].border = border
                row += 1
        ws.column_dimensions['A'].width = 15
        ws.column_dimensions['B'].width = 20
        ws.column_dimensions['C'].width = 59
    wb.save(path)
    return path

print("✅ 函數載入完成")

✅ 函數載入完成


## 🚀 執行爬蟲

In [9]:
start_dt = datetime.strptime(START_DATE, "%Y-%m-%d").date() if USE_DATE_FILTER else None
end_dt = datetime.strptime(END_DATE, "%Y-%m-%d").date() if USE_DATE_FILTER else None

driver = setup_driver()
wait = WebDriverWait(driver, 10)

try:
    print("🔐 登入中...")
    driver.get('http://www.landsen.com.tw/lsdoc/login.php')
    wait.until(EC.presence_of_element_located((By.NAME, 'get_acc'))).send_keys(USERNAME)
    wait.until(EC.presence_of_element_located((By.NAME, 'get_pawd'))).send_keys(PASSWORD)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@type="SUBMIT"]'))).click()
    print("✅ 登入成功")

    driver.get('http://www.landsen.com.tw/lsdoc/index.php?pid=DOC04')
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'list_table')))

    if HANDLER_NAME:
        print(f"🔍 查詢承辦人: {HANDLER_NAME}")
        h_input = wait.until(EC.presence_of_element_located((By.NAME, 'get_hand_emp')))
        h_input.clear()
        h_input.send_keys(HANDLER_NAME)
        btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn_search')))
        btn.click()
        wait.until(EC.staleness_of(btn))
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'list_table')))

    print("📥 抓取收文...")
    doc_ids = get_links(driver, NUM_DOCS)
    records = []

    for i, did in enumerate(doc_ids, 1):
        print(f"  處理收文 {i}/{len(doc_ids)}", end='\r')
        info = extract_info(driver, did)
        if info and info.get('公文日期'):
            if USE_DATE_FILTER and not check_date(info['公文日期'], start_dt, end_dt):
                continue
            records.append({'type':'收文', 'date':info['公文日期'], 'number':info['公文字號'],
                           'subject':info['主旨摘要'], 'site':info['工地別']})
    print(f"\n✅ 收文: {len(doc_ids)}筆")

    print("📤 切換發文頁...")
    driver.get('http://www.landsen.com.tw/lsdoc/index.php?pid=DOC04')
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'list_table')))

    try:
        fa_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='發　文']")))
        fa_btn.click()
        wait.until(EC.staleness_of(fa_btn))
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'list_table')))

        if HANDLER_NAME:
            h_input = wait.until(EC.presence_of_element_located((By.NAME, 'get_hand_emp')))
            h_input.clear()
            h_input.send_keys(HANDLER_NAME)
            btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn_search')))
            btn.click()
            wait.until(EC.staleness_of(btn))
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'list_table')))

        fa_ids = get_links(driver, NUM_DOCS)
        for i, did in enumerate(fa_ids, 1):
            print(f"  處理發文 {i}/{len(fa_ids)}", end='\r')
            info = extract_info(driver, did, True)
            if info and info.get('日期'):
                if USE_DATE_FILTER and not check_date(info['日期'], start_dt, end_dt):
                    continue
                records.append({'type':'發文', 'date':info['日期'], 'number':info['發文字號'],
                               'subject':info['主旨'], 'site':info['工地別']})
        print(f"\n✅ 發文: {len(fa_ids)}筆")
    except Exception as e:
        print(f"⚠️ 發文錯誤: {e}")

    if records:
        print("📊 生成Excel...")
        data = defaultdict(list)
        for r in records:
            data[r['site']].append(r)
        excel_path = make_excel(data)
        print(f"🎉 完成! 共{len(records)}筆, {len(data)}個工地")
        from google.colab import files
        files.download(excel_path)
    else:
        print("⚠️ 沒有資料")

except Exception as e:
    print(f"❌ 錯誤: {e}")
finally:
    driver.quit()

🔐 登入中...
❌ 錯誤: HTTPConnectionPool(host='localhost', port=39419): Read timed out. (read timeout=120)


ConnectionResetError: [Errno 104] Connection reset by peer